In [1]:
#Preprocessing

In [2]:
import pandas as pd
import numpy as np

import os
directory = f'/Users/lilimatic/greenspace'
os.chdir(directory)

def stats(col):
    features = list(set(', '.join(map(str, df[col].to_numpy())).split(', ')))
    stats = pd.DataFrame(np.array([[int(feature in str(item)) for item in df[col].to_numpy()] for feature in features]).T, columns=features,
                        index=df.index)
    return stats

df = pd.read_csv(f'data/data.csv',index_col=0)

#Clean data set and read

df = df.loc[:, df.columns!='Timestamp']
df = df[df['Gender'] != 'Apache attack helicopter']
df = df[df['If your identity was not revealed, would you want to provide your phone’s location (see question 10)?'] != 'Yes']

df = df.dropna()

#Dictionary to understand what is where 
rename = pd.Series(list(df.columns),index=['age','gender','municipality','precovid', 'freqency','time', 'activity','socializing','distancing',
'physicalsafety','covidsafety', 'interest','location']).to_dict()

df.columns = ['age','gender','municipality','precovid', 'frequency','time', 'activity','socializing','distancing',
'physicalsafety','covidsafety', 'interest','location']

In [3]:
df = df.reset_index(drop=True)

In [4]:
ages = {'7-14': 12.4, '15-24': 19.5, '25-40': 32.5, '41-64': 52.5}
df['age'] = df['age'].apply(lambda x: ages[x])

minutes = {
    'Not at all': 0,
    'Up to 15 minutes': 0.25,
    'From 15 to 30 minutes': 0.5,
    'From 30 minutes to one hour': 0.75,
    'More than one hour': 1.0,
}

df['time'] = df['time'].apply(lambda x: minutes[x])





In [5]:
df['safety'] = df.loc[:, 'physicalsafety']

In [6]:
df[df.physicalsafety != df.covidsafety].index

Int64Index([  3,  20,  27,  29,  60,  62,  65,  72,  82,  83,  88,  93, 107,
            122, 138, 139, 164, 166, 172, 177, 183, 184, 185, 207, 225, 230,
            232, 235, 236, 239, 244, 247, 251, 252, 256, 263, 269, 274, 275,
            287, 296, 302, 304, 309, 314, 318, 319, 322, 323, 324, 325, 327,
            328, 330, 332, 335, 336, 337, 338, 339, 340, 342, 349, 351, 352,
            355, 358, 360, 361, 363, 365, 367, 373, 374, 375, 377, 378, 381,
            386, 387, 394, 398],
           dtype='int64')

In [7]:
df.safety[df[df.physicalsafety != df.covidsafety].index] = 0

<ipython-input-7-14b50573ff2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.safety[df[df.physicalsafety != df.covidsafety].index] = 0
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [8]:
hotone = ['gender','distancing', 'interest','precovid','frequency','location','safety'] #'physicalsafety','covidsafety',

df = pd.get_dummies(df,columns=hotone)

df = pd.concat([df,stats('municipality'),stats('activity'),stats('socializing')],axis=1)

df = df.drop(columns=['municipality', 'activity','socializing','physicalsafety','covidsafety']) 

In [9]:
df

,age,time,gender_Female,gender_Male,distancing_No,distancing_Yes,interest_No,interest_Yes,"precovid_Always, almost every day",precovid_Not at all,...,Physical activity,Walking the dog,Consumption,Sitting and relaxing,Spending time with friends,With a large group of friends (more than 5 people),With children,With a small group of friends (up to 5 persons),Alone or with pets,With partner or friend (two adults)
0,19.5,1.00,0,1,1,0,0,1,0,0,...,1,0,0,1,1,0,0,1,0,0
1,32.5,0.50,0,1,0,1,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
2,52.5,0.50,1,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,0,1,0
3,19.5,0.75,1,0,0,1,0,1,0,0,...,0,1,1,1,1,0,0,1,1,1
4,52.5,0.25,1,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,52.5,0.25,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
396,19.5,0.75,1,0,0,1,0,1,1,0,...,0,0,0,1,0,0,0,0,0,1
397,19.5,0.75,1,0,0,1,0,1,1,0,...,0,1,0,0,1,0,0,0,0,1
398,19.5,0.75,1,0,0,1,0,1,1,0,...,1,0,0,0,0,0,0,1,0,0
